In [ ]:
import signal
from pathlib import Path

import wget

parser_version = '1.6.7'
url = f'https://github.com/nemoware/document-parser/releases/download/{parser_version}/document-parser-{parser_version}.jar'
if not Path(f'document-parser-{parser_version}.jar').is_file():
    wget.download(url)

In [ ]:
import glob
import zipfile

with zipfile.ZipFile(f'./{glob.glob("ДД по практикам*.zip")[0]}',
                     'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
parser_version = '1.6.7'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "Документы\ДД по практикам\Практика правового сопровождения закупок МТР и услуг общего профиля\ДД ООО И.doc"

# Start

## Import

In [ ]:
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time
import importlib
import logging
import search_text
import search_text_v2

## Create list of all docs path

In [ ]:
arrOfAllDocs = []

for root, dir_names, filenames in os.walk('Документы'):
    if len(root.split('\\')) == 2: continue
    flag = False
    for i in root.split('\\'):
        if str(i).startswith('Исключена'):
            flag = True
            break
    if flag: continue
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))


In [ ]:
arrOfAllDocs[:]

In [ ]:
import multiprocessing


def driver_func(documents, path):
    processes = 4
    with multiprocessing.Pool(processes) as pool:
        params = zip(documents, path)
        print(params)
        results = [pool.apply_async(search_text_v2.get_text, param) for param in params]

        for r in results:
            print('\t', r.get())

In [ ]:
importlib.reload(search_text)
importlib.reload(search_text_v2)

index = 1
number_of_docs = 0
parser_version = '1.6.7'

labels = ['Практика коммерческой логистики',
          'Практика недропользования и экологии',
          'Практика поддержки региональных, розничных продаж и клиентского сервиса',
          'Практика правового сопровождения закупок МТР и услуг общего профиля',
          'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
          'Практика правового сопровождения операционной деятельности БРД',
          'Практика правового сопровождения переработки и инфраструктуры',
          'Практика правовой поддержки брендов',
          'Практика правовой поддержки использования и коммерциализации ИС',
          'Практика правовой поддержки создания и приобретения ИС',
          'Практика промышленной безопасности и охраны труда',
          'Практика финансового и конкурентного права',
          'Практика экспорта, оптовых продаж и сбыта бизнес-единиц (БЕ)']

result = []
result_of_fail = []
result_of_possible = []
result_of_possible2 = []

s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

parser_url = "http://localhost:8083/"
java_subprocess = None
try:
    i = 1
    try:
        print(f"Проверка, если парсер заупущен на {parser_url}")
        response = requests.post(
            f"{parser_url}status",
            headers=headers
        )
    except Exception as e:
        print(
            "Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс"
        )
        java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                           stdout=subprocess.PIPE, encoding="utf-8")
        time.sleep(2)
        while True:
            # time.sleep(0.1)
            output_log_spring = java_subprocess.stdout.readline()
            sys.stdout.write("\rПроверка соединения")
            sys.stdout.flush()
            i += 1
            if output_log_spring.find("Started DocumentParserService") != -1:
                print("\nГотово")
                java_subprocess.stdout.close()
                break
            if i > 31:
                raise Exception("Не удалось получить доступ к ранее запущенному парсеру")
    print("Запустился успешно")
    print("Общее количество документов =", len(arrOfAllDocs))

    for docs in arrOfAllDocs:
        try:
            file = open(docs, 'rb')
            encoded_string = base64.b64encode(file.read())
            encoded_string = str(encoded_string)[2:-1]
        except Exception as e:
            # print(f"\nОшибка в файле {docs}")
            # print(f"при конвертации в base64, исключение = {e.msg}")
            # print("=" * 200)
            continue
        is_doc = True
        is_docx = True
        is_not_bad_doc = True
        doc_type = docs.split(".")[-1].upper()
        resArr = []
        document = []
        while is_doc or is_docx or is_not_bad_doc:
            response = requests.post(
                f"{parser_url}document-parser",
                data=json.dumps({
                    "base64Content": encoded_string,
                    "documentFileType": doc_type
                }),
                headers=headers
            )
            if 'message' in response.json() and not (is_doc == False and is_docx == False):
                if doc_type == 'DOC':
                    is_doc = False
                    doc_type = 'DOCX'
                    continue
                if doc_type == 'DOCX':
                    is_docx = False
                    doc_type = 'DOC'
                    continue

            try:
                resArr = response.json()['documents']
            except Exception as e:
                # print(f"\nОшибка в файле {docs}")
                # print(f"Ответ от парсера {response.json()}")
                # print("=" * 200)
                continue
            finally:
                is_doc = False
                is_docx = False
                is_not_bad_doc = False

        sys.stdout.write(
            f"\rNow {index} from {len(arrOfAllDocs)} и Good = {len(result)}, Bad = {len(result_of_fail)}, TEST = {len(result_of_possible)}, TEST2 = {len(result_of_possible2)}")
        sys.stdout.flush()
        index += 1

        if not resArr:
            continue
        # driver_func(resArr, docs)
        for document in resArr:
            # result_of_single_doc, enum = search_text.find_text(document, path=docs)
            result_of_single_doc, enum = search_text_v2.get_text(document, path=docs)

            if enum == search_text_v2.list_of_sheets.GOOD:
                result.append(result_of_single_doc)
            if enum == search_text_v2.list_of_sheets.BAD:
                result_of_fail.append(result_of_single_doc)
            if enum == search_text_v2.list_of_sheets.BAD2:
                result_of_possible.append(result_of_single_doc)
            if enum == search_text_v2.list_of_sheets.GOOD2:
                result.append(result_of_single_doc)
                result_of_possible2.append(result_of_single_doc)

    with pd.ExcelWriter("classifier.xlsx", engine="xlsxwriter",
                        engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        # writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

        df = pd.DataFrame(result)
        df = df.drop_duplicates(subset=['text'])
        df.to_excel(writer, 'good', engine='xlsxwriter')
        sheets_good = writer.sheets['good']
        sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

        df = pd.DataFrame(result_of_fail)
        df.to_excel(writer, 'SO SO', engine='xlsxwriter')
        sheets_bad = writer.sheets['SO SO']
        sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])
        logging.info("\nФайл создан")
except Exception as e:
    logging.error(e)
finally:
    #Смерть java процессу!
    if java_subprocess is not None:
        if platform.system() == 'Windows':
            subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
        elif platform.system() == 'Linux':
            os.kill(java_subprocess.pid, signal.SIGTERM)
        else:
            print('Не известная платформа, убейте в ручную процесс java')
# Now 2277 from 2277 и Good = 2970, Bad = 52, TEST = 846, TEST2 = 2053

In [ ]:
df = pd.read_excel(open('classifier.xlsx', 'rb'), sheet_name='good', index_col=0)

In [ ]:
df[df['text'].apply(lambda x: not isinstance(x, str))]